In [43]:
import math
outputs = [0, 0, 2, 2, 1] # X_1 .. X_N
transition_prob = [[0.55, 0.25, 0.2],\
                   [0.15, 0.7, 0.15],\
                   [0.2, 0.3, 0.4]]
emission_prob = [[0.1, 0.3, 0.6],\
                 [0.2, 0.5, 0.3],\
                 [0.6, 0.3, 0.1]]
start_prob = [0.3, 0.3, 0.4]

In [45]:
import math

class HMM:
  def __init__(self, transition_prob: list[list[int]], emission_prob: list[list[int]], start: list[int]):
    # P(Y_(t+1) | Y_t)
    self.transition_prob = transition_prob

    # P(X_t | Y_t)
    self.emission_prob = emission_prob

    self.start = start

  def joint_prob(self, outputs: list[int], hiddens: list[int]) -> list[float]:
    if len(outputs) != len(hiddens):
        raise ValueError("Output and hidden sequences must have the same length")
    
    joint_p = self.start[hiddens[0]] * self.emission_prob[hiddens[0]][outputs[0]]
    
    for t in range(1, len(outputs)):
        transition_p = self.transition_prob[hiddens[t-1]][hiddens[t]]
        emission_p = self.emission_prob[hiddens[t]][outputs[t]]
        joint_p *= transition_p * emission_p
    
    return joint_p

  def conditional_weights(self, outputs: list[int], hiddens: list[int], t: int) -> float:
    n = len(outputs)
    num_states = 3  # Number of possible states (0, 1, 2)
    weights = [0.0] * num_states
    
    for y in range(num_states):
        weight = 1.0
        if t == 0:
            weight *= self.start[y]
        else:
            weight *= self.transition_prob[hiddens[t-1]][y]
        
        if t < n - 1:
            weight *= self.transition_prob[y][hiddens[t+1]]
        
        weight *= self.emission_prob[y][outputs[t]]
        
        weights[y] = weight
    
    return weights

In [46]:
hmm = HMM(transition_prob, emission_prob, start_prob)

In [47]:
# Test cases
hmm.joint_prob([0,0,2,2,1], [1,1,1,1,1])
# about 0.000129654


0.00012965399999999996

In [48]:
hmm.joint_prob([0,0,2,2,1], [2,2,0,0,1])
# 0.00028512

0.00028512

In [49]:
import itertools

def get_most_likely_hidden_states(hmm: HMM, outputs: list[int]):
  joint_probs = {}
    
  # iterate over possible hidden states
  for hiddens in itertools.product([0, 1, 2], repeat=len(outputs)):
      hiddens_list = list(hiddens)  # convert to list
      prob = hmm.joint_prob(outputs, hiddens_list)
      joint_probs[hiddens] = prob
  
  # Calculate P(X_1,...,X_n) by summing P(Y_1,...,Y_n,X_1,...,X_n) over all possible values of Y
  evidence = sum(joint_probs.values())
  
  # Calculate conditional probabilities P(Y_1,...,Y_n|X_1,...,X_n)
  conditional_probs = {hiddens: prob / evidence for hiddens, prob in joint_probs.items()}
  
  # Find the sequence with the highest conditional probability
  max_prob = -1
  best_hiddens = None
  
  for hiddens, prob in conditional_probs.items():
      if prob > max_prob:
          max_prob = prob
          best_hiddens = list(hiddens)  # convert tuple to list
  
  # Get the top 10 sequences by conditional probability
  top_sequences = sorted(conditional_probs.items(), key=lambda x: x[1], reverse=True)[:10]
  
  for i, (hiddens, prob) in enumerate(top_sequences):
      # Get the joint probability for this sequence
      joint_prob = joint_probs[hiddens]
      print(f"{i+1}. Sequence: {list(hiddens)}, Conditional Prob: {prob}, Joint Prob: {joint_prob}")
  
  return best_hiddens, max_prob


In [50]:
get_most_likely_hidden_states(hmm, outputs)

1. Sequence: [2, 2, 1, 1, 1], Conditional Prob: 0.10142053368734855, Joint Prob: 0.00038102399999999994
2. Sequence: [2, 2, 0, 0, 0], Conditional Prob: 0.10017864960138104, Joint Prob: 0.0003763584
3. Sequence: [2, 2, 0, 0, 1], Conditional Prob: 0.0758929163646826, Joint Prob: 0.00028512
4. Sequence: [2, 1, 1, 1, 1], Conditional Prob: 0.05916197798428665, Joint Prob: 0.00022226399999999996
5. Sequence: [2, 2, 0, 1, 1], Conditional Prob: 0.04829549223207073, Joint Prob: 0.00018143999999999997
6. Sequence: [2, 2, 0, 0, 2], Conditional Prob: 0.03642859985504764, Joint Prob: 0.00013685759999999998
7. Sequence: [1, 1, 1, 1, 1], Conditional Prob: 0.03451115382416721, Joint Prob: 0.00012965399999999996
8. Sequence: [2, 0, 0, 0, 0], Conditional Prob: 0.02295760720031649, Joint Prob: 8.624880000000002e-05
9. Sequence: [2, 2, 1, 0, 0], Conditional Prob: 0.0204910874184643, Joint Prob: 7.698239999999999e-05
10. Sequence: [2, 2, 2, 1, 1], Conditional Prob: 0.019318196892828304, Joint Prob: 7.25760

([2, 2, 1, 1, 1], 0.10142053368734855)

# Gibbs Sampling

In [51]:
# Test cases
print(hmm.conditional_weights([0,0,2,2,1], [1,1,1,1,1], 2))
# about [0.0225, 0.147, 0.0045]
print(hmm.conditional_weights([0,0,2,2,1], [2,2,0,0,1], 2))
# about [0.066, 0.0135, 0.00800000000000000]

[0.0225, 0.14699999999999996, 0.0045]
[0.066, 0.0135, 0.008000000000000002]


In [52]:
# Perform a single Gibbs sample Y_1 .. Y_N ~ P(Y_1 .. Y_N | X_1 .. X_N)
import random

def gibbs_sample(hmm: HMM, outputs: list[int]) -> list[list[int]]:
  n = len(outputs)
  num_states = len(hmm.start)
  
  # Initialize with a random sequence
  hiddens = [random.choice(range(num_states)) for _ in range(n)]
  
  # Perform one Gibbs sweep through all positions
  for t in range(n):
      weights = hmm.conditional_weights(outputs, hiddens, t)

      hiddens[t] = random.choices(population=range(num_states), weights=weights, k=1)[0]
  
  return hiddens

In [53]:
import collections
def estimate_likely_hidden_states(hmm: HMM, outputs: list[int]):
  hiddens = [random.choice(range(len(hmm.start))) for _ in range(len(outputs))]
  for _ in range(1000):  # 1000 Warm-up iterations on outer
      hiddens = gibbs_sample(hmm, outputs)
  
  # Collect samples
  samples = []
  for _ in range(5000):  # 5000 actual sampling iterations
      hiddens = gibbs_sample(hmm, outputs)
      samples.append(tuple(hiddens))
  
  counter = collections.Counter(samples)
  most_common = counter.most_common(10)
  
  # Calculate joint probabilities for the most common sequences
  result = []
  for sequence, count in most_common:
      sequence_list = list(sequence)
      joint_prob = hmm.joint_prob(outputs, sequence_list)
      result.append((sequence_list, joint_prob, count/5000))
  
  result.sort(key=lambda x: x[1], reverse=True)
  
  for i, (seq, prob, freq) in enumerate(result):
      print(f"{i+1}. Sequence: {seq}, Probability: {prob}, Frequency: {freq}")
  
  return [seq for seq, _, _ in result]

In [54]:
estimate_likely_hidden_states(hmm, outputs)

1. Sequence: [2, 2, 1, 1, 1], Probability: 0.00038102399999999994, Frequency: 0.0812
2. Sequence: [2, 2, 0, 0, 0], Probability: 0.0003763584, Frequency: 0.0848
3. Sequence: [2, 2, 0, 0, 1], Probability: 0.00028512, Frequency: 0.073
4. Sequence: [2, 1, 1, 1, 1], Probability: 0.00022226399999999996, Frequency: 0.0404
5. Sequence: [2, 2, 0, 1, 1], Probability: 0.00018143999999999997, Frequency: 0.04
6. Sequence: [2, 2, 0, 0, 2], Probability: 0.00013685759999999998, Frequency: 0.0342
7. Sequence: [1, 1, 1, 1, 1], Probability: 0.00012965399999999996, Frequency: 0.029
8. Sequence: [2, 2, 1, 0, 0], Probability: 7.698239999999999e-05, Frequency: 0.0302
9. Sequence: [2, 2, 2, 1, 1], Probability: 7.257600000000002e-05, Frequency: 0.0242
10. Sequence: [2, 2, 2, 0, 0], Probability: 4.561920000000001e-05, Frequency: 0.0248


[[2, 2, 1, 1, 1],
 [2, 2, 0, 0, 0],
 [2, 2, 0, 0, 1],
 [2, 1, 1, 1, 1],
 [2, 2, 0, 1, 1],
 [2, 2, 0, 0, 2],
 [1, 1, 1, 1, 1],
 [2, 2, 1, 0, 0],
 [2, 2, 2, 1, 1],
 [2, 2, 2, 0, 0]]